In [1]:
import os

In [2]:
%pwd

'c:\\Users\\Sanju\\WORKSPACE\\Late-Delivery-Classification-Machine-Learning-Project\\research'

In [3]:
os.chdir('../')

In [4]:
%pwd

'c:\\Users\\Sanju\\WORKSPACE\\Late-Delivery-Classification-Machine-Learning-Project'

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataValidationConfig:
    root_dir: Path
    unzip_data_dir: Path
    STATUS_FILE: str
    train_data_path: Path
    test_data_path: Path
    all_schema: dict

In [6]:
from MLProject.constants import *
from MLProject.utils.common import read_yaml, create_directories

In [7]:
class ConfigurationManager:
        def __init__(
                self,
                config_filepath = CONFIG_FILE_PATH,
                params_filepath = PARAMS_FILE_PATH,
                schema_filepath = SCHEMA_FILE_PATH):
        
                self.config = read_yaml(config_filepath)
                self.params = read_yaml(params_filepath)
                self.schema = read_yaml(schema_filepath)

                create_directories([self.config.artifacts_root])

        def get_data_validation_config(self) -> DataValidationConfig:
                config = self.config.data_validation
                schema = self.schema.COLUMNS


                create_directories([config.root_dir])

                data_validation_config = DataValidationConfig(
                        root_dir=config.root_dir,
                        unzip_data_dir=config.unzip_data_dir,
                        STATUS_FILE=config.STATUS_FILE,
                        train_data_path=config.train_data_path,
                        test_data_path=config.test_data_path,
                        all_schema=schema

                )

                return data_validation_config

In [8]:
import os
from MLProject import logger
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [9]:
class DataValidation:
    def __init__(self, config: DataValidationConfig):
        self.config = config

    def validate_all_columns(self)->bool:
        try:
            validation_status = None

            data = pd.read_csv(self.config.unzip_data_dir)

            all_cols = list(data.columns)

            all_schema = self.config.all_schema.keys()


            for col in all_cols:
                if col not in all_schema:
                    validation_status = False
                    with open(self.config.STATUS_FILE, 'w') as f:
                        f.write(f'Validation status: {validation_status}')

                else:
                    validation_status = True
                    with open(self.config.STATUS_FILE, 'w') as f:
                        f.write(f'Validation status: {validation_status}')

            return validation_status
        
        except Exception as e:
            raise e
        
    
    def initiate_data_split(self):
        try:
            df = pd.read_csv(self.config.unzip_data_dir)

            train_set, test_set = train_test_split(df, test_size=0.2, random_state=42)

            train_set.to_csv(self.config.train_data_path, index=False, header=True)
            test_set.to_csv(self.config.test_data_path, index=False, header=True)

            return(
                self.config.train_data_path, 
                self.config.test_data_path
            )
        
        except Exception as e:
            raise e

In [10]:
try:
    config = ConfigurationManager()
    data_validation_config = config.get_data_validation_config()
    data_validation = DataValidation(config=data_validation_config)
    data_validation.validate_all_columns()
    data_validation.initiate_data_split()
except Exception as e:
    raise e

[2024-02-29 15:53:13,693: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-02-29 15:53:13,695: INFO: common: yaml file: params.yaml loaded successfully]
[2024-02-29 15:53:13,698: INFO: common: yaml file: schema.yaml loaded successfully]
[2024-02-29 15:53:13,699: INFO: common: created directory at: artifacts]
[2024-02-29 15:53:13,700: INFO: common: created directory at: artifacts/data_validation]
